In [1]:
# TODO: add crop for log_sigma and for abs diff for FVC
# TODO: add normalization for data
# TODO: ...

In [2]:
%config Completer.use_jedi = False

import os
import platform
from collections import namedtuple

# import tqdm
import pandas as pd
import numpy as np
import sparse

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.nn.modules.loss import _Loss
from torch.optim.lr_scheduler import _LRScheduler
# from torchvision import transforms
# from torchsummary import summary
# from efficientnet_pytorch_3d import EfficientNet3D
from my_efficientnet_pytorch_3d import EfficientNet3D

# from utils import *


########################

RUNNING_IN_KAGGLE = 'linux' in platform.platform().lower()
IMAGE_PATH = "../input/osic-pulmonary-fibrosis-progression/" if RUNNING_IN_KAGGLE else 'data/'
PROCESSED_PATH = 'FIX IT!' if RUNNING_IN_KAGGLE else 'data/processed-data/'  # TODO: fix this line

dtype = torch.float32
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
device = torch.device(device)

In [3]:
class CTDataset(Dataset):
    _ReturnValue = namedtuple('ReturnValue', ['weeks', 'fvcs', 'features', 'masks', 'images'])

    def __init__(
            self, root, csv_path, train=True, test_size=0.25, random_state=42):
        """
        :param dataset:

        :param root:
        :param train:
        :param train_test_split:
        :param random_state:
        """
        assert test_size is not None

        self.root = root
        self.train = train
        self.csv_path = csv_path
        self.test_size = test_size
        self.random_state = random_state

        if not os.path.exists(self.root):
            raise ValueError('Data is missing')

        self._patients = list(sorted(os.listdir(self.root)))

        if self.test_size == 0:
            self._train_patients, self._test_patients = self._patients, []
        else:
            self._train_patients, self._test_patients = train_test_split(
                self._patients, test_size=self.test_size, random_state=random_state
            )

        self._table_features = dict()
        table_data = pd.read_csv(self.csv_path)
        for patient in self._patients:
            patient_data = table_data[table_data.Patient == patient]

            all_weeks = patient_data.Weeks.tolist()
            all_fvcs = patient_data.FVC.tolist()

            all_weeks, all_fvcs = zip(*sorted(zip(all_weeks, all_fvcs), key=lambda x: x[0]))

            age = sorted(zip(*np.unique(patient_data.Age, return_counts=True)), key=lambda x: x[1])[-1][0]
            sex = sorted(zip(*np.unique(patient_data.Sex, return_counts=True)), key=lambda x: x[1])[-1][0]
            smoking_status = sorted(zip(*np.unique(patient_data.SmokingStatus, return_counts=True)), key=lambda x: x[1])[-1][0]

            sex = [0, 1] if sex == 'Female' else [1, 0]
            smoking_status = (
                [1, 0, 0] if smoking_status == 'Ex-smoker' else
                [0, 1, 0] if smoking_status == 'Never smoked' else
                [0, 0, 1] if smoking_status == 'Currently smokes' else
                [0, 0, 0]
            )
            self._table_features[patient] = (
                all_weeks, all_fvcs, [age] + sex + smoking_status
            )

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        patient = self._train_patients[index] if self.train else self._test_patients[index]
        base_path = os.path.join(self.root, patient)

        meta = np.load(os.path.join(base_path, 'meta.npy'), allow_pickle=True).tolist()
        masks = sparse.load_npz(os.path.join(base_path, 'masks.npz'))
        images = np.load(os.path.join(base_path, 'images.npy'))

        meta_processed = dict()
        for key, values in meta.items():
            if key in {'SliceLocation', 'InstanceNumber'}:
                continue
            else:
                unique_values, values_cnt = np.unique(values, return_counts=True, axis=0)
                most_frequent = sorted(zip(unique_values, values_cnt), key=lambda x: x[1])[-1][0]
                most_frequent = np.array(most_frequent).reshape(-1)
                if key in {
                    'SliceThickness', 'TableHeight', 'WindowCenter', 'WindowWidth'
                }:
                    meta_processed[key] = most_frequent[0]
                elif key == 'PixelSpacing':
                    if len(most_frequent) == 1:
                        meta_processed['PixelSpacingX'], meta_processed['PixelSpacingY'] = (
                            most_frequent[0], most_frequent[0]
                        )
                    else:
                        meta_processed['PixelSpacingX'], meta_processed['PixelSpacingY'] = (
                            most_frequent[0], most_frequent[1]
                        )
                elif key == 'PatientPosition':
                    pass
                elif key == 'PositionReferenceIndicator':
                    pass

        all_weeks, all_fvcs, features = self._table_features[patient]
        features = [value for key, value in meta_processed.items()] + features

        return CTDataset._ReturnValue(weeks=all_weeks, fvcs=all_fvcs, features=features, masks=masks, images=images)

    def __len__(self):
        return len(self._train_patients if self.train else self._test_patients)

    def __repr__(self):
        fmt_str = 'OSIC Pulmonary Fibrosis Progression Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        tmp = 'train' if self.train is True else 'test'
        fmt_str += '    Split: {}\n'.format(tmp)
        fmt_str += '    Root Location: {}\n'.format(self.root)
        return fmt_str


class LaplaceLoss(_Loss):
    def forward(self, y_true, preds, log_sigma):
        losses = np.sqrt(2) * (y_true - preds).abs() / log_sigma.exp() + log_sigma + np.log(2) / 2
        return losses.mean()


class SqueezeLayer(nn.Module):
    def forward(self, x):
        return x.squeeze()


class FeatureExtractor(nn.Module):
    def __init__(self, net):
        super().__init__()
        self.net = net

    def forward(self, x):
        return self.net.extract_features(x.unsqueeze(0).unsqueeze(0))


class OSICNet(nn.Module):
    def __init__(self, dtype, device, efficient_net_model_number, hidden_size, dropout_rate):  # , output_size
        super().__init__()

        self.dtype = dtype
        self.device = device

        self.CT_features_extractor = nn.Sequential(
            FeatureExtractor(
                EfficientNet3D.from_name(
                    f'efficientnet-b{efficient_net_model_number}', override_params={'num_classes': 1}, in_channels=1
                )
            ),
            nn.AdaptiveAvgPool3d(1),
            SqueezeLayer()
        )

        self.predictor = nn.Sequential(
            nn.Linear(1294, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, 5)  # output_size
        )

        self.CT_features_extractor.to(self.device)
        self.predictor.to(self.device)

    def forward(self, data):
        lungs = -1000 * (1.0 - data.masks) + data.masks * data.images
        lungs = torch.tensor(lungs, dtype=self.dtype, device=self.device)
        lungs_features = self.CT_features_extractor(lungs)

        data_weeks = torch.tensor(data.weeks, dtype=self.dtype)
        weeks = torch.empty(len(data.weeks), 4, dtype=self.dtype)
        weeks[:, 3] = 1
        weeks[:, 2] = data_weeks
        weeks[:, 1] = data_weeks ** 2
        weeks[:, 0] = data_weeks ** 3

        agg_loss = 0
        for week, FVC in zip(data.weeks, data.fvcs):
            table_features = torch.tensor(np.r_[week, FVC, data.features], dtype=self.dtype, device=self.device)
            X = torch.cat([lungs_features, table_features])

            pred_numbers = self.predictor(X).cpu()
            coefs = pred_numbers[:4]
            log_sigma = pred_numbers[4]

            FVC_preds = (weeks * coefs).sum(dim=1)
            FVC_true = torch.tensor(data.fvcs, dtype=self.dtype)
            
            agg_loss += LaplaceLoss()(FVC_true, FVC_preds, log_sigma)

        return agg_loss / len(data.weeks)


class LinearDecayLR(_LRScheduler):
    def __init__(self, optimizer, start_epoch, stop_epoch, start_lr, stop_lr, last_epoch=-1):
        self.optimizer = optimizer

        self.start_epoch = start_epoch
        self.stop_epoch = stop_epoch

        self.start_lr = start_lr
        self.stop_lr = stop_lr

        self.last_epoch = last_epoch

        super().__init__(optimizer, last_epoch)

    def get_lr(self) -> list:
        if self.last_epoch < self.start_epoch:
            new_lr = self.start_lr
        elif self.last_epoch > self.stop_epoch:
            new_lr = self.stop_lr
        else:
            new_lr = self.start_lr + (
                (self.stop_lr - self.start_lr) *
                (self.last_epoch - self.start_epoch) /
                (self.stop_epoch - self.start_epoch)
            )
        return [new_lr for _ in self.optimizer.param_groups]

In [4]:
train_dataset = CTDataset(
    f'{PROCESSED_PATH}/train',
    f'{IMAGE_PATH}/train.csv',
    train=True, test_size=0.25, random_state=42
)

test_dataset = CTDataset(
    f'{PROCESSED_PATH}/train',
    f'{IMAGE_PATH}/train.csv',
    train=False, test_size=0.25, random_state=42
)

MAX_EPOCHS = 1
model = OSICNet(dtype=dtype, device=device, efficient_net_model_number=0, hidden_size=512, dropout_rate=0.5)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
# lr_scheduler = LinearDecayLR(optimizer, )

In [5]:
for epoch in range(MAX_EPOCHS):
    for cur_iter, data in enumerate(train_dataset):  # tqdm(, desc='Iteration over dataset'):
        optimizer.zero_grad()
        loss = model(data)
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1:3d}, iter {cur_iter + 1:4d}, loss {loss.item():10.3f}')

Epoch   1, iter    1, loss  67196.227


RuntimeError: CUDA out of memory. Tried to allocate 1.24 GiB (GPU 0; 11.00 GiB total capacity; 6.09 GiB already allocated; 1.24 GiB free; 7.49 GiB reserved in total by PyTorch)